# 2019.09.28 문제 복기

  - Yelp.com 리뷰 데이터(text)를 활용한 별점 예측
    데이터 셋 소스 : kaggle (https://www.kaggle.com/omkarsabnis/yelp-reviews-dataset)

In [3]:
## pandas, numpy import 하기

import pandas as pd
import numpy as np

In [7]:
## 파일 읽기

df = pd.read_csv("data/yelp.csv")

In [10]:
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [12]:
df.shape

(10000, 10)

## 4. 별점 1,5인 값만 뽑아서 CountVectorizer

  1). 전체 voca 수
  2) x_train 특정 x 벡터화 시 처음으로 1이 나오는 단어
  3) love가 몇번째 피쳐인지?

In [19]:
## 4번문제를 위한 전처리
## 별점이 1,5 인것만 뽑아서 df1 로 저장

df1 = df[(df['stars']==1) | (df['stars']==5)]
df1.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0
6,zp713qNhx8d9KCJJnrw1xA,2010-02-12,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doing and drive here. After I...,review,wFweIWhv2fREZV_dYkz_1g,7,7,4


In [22]:
df1.groupby("stars").count()

,business_id,date,review_id,text,type,user_id,cool,useful,funny
stars,,,,,,,,,
1,749,749,749,749,749,749,749,749,749
5,3337,3337,3337,3337,3337,3337,3337,3337,3337


In [23]:
X = df1['text']
y = df1['stars']

In [24]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3064,)
(1022,)
(3064,)
(1022,)


In [141]:
# CountVectorizer 로드
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words='english')
##vect = CountVectorizer()
##stopwords 안했더니 뒤에 평점 1, 5 인 경우 빈도 높은 단어가 a, the 같은게 나와서 추가함..

In [142]:
## 데이터 벡터로 변환
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

### (위의 동작과 동일) `fit_transform`: 어휘 학습 및 벡터화를 동시에 수행 
#X_train_dtm = vect.fit_transform(X_train)

In [140]:
CountVectorizer?

In [143]:
X_train_dtm

<3064x16528 sparse matrix of type '<class 'numpy.int64'>'
	with 143743 stored elements in Compressed Sparse Row format>

In [145]:
## DataFrame 형태로 변환한다
train_vector = pd.DataFrame(X_train_dtm.toarray(), columns=vect.get_feature_names())
train_vector.head()

,00,000,00a,00am,00pm,01,02,03,03342,04,...,zucchini,zuchinni,zumba,zupa,zuzu,zwiebel,zzed,éclairs,école,ém
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


만약 컬럼을 다 보고 싶다면!

pd.set_option("display.max_columns",None)

위 코드를 입력하면 되나, 시험때는 비추 
(저 옵션을 넣게 되면 시험 때, 컬럼을 다 가져오느라 로딩 시간이 길어지게 된다. 1초도 아까우니까)

### 4-1). 전체 voca 수


In [35]:
##shape로 구할 수 있음. 결국 전체 voca수는 feature 수 이기 때문 따라서 답은 16825
train_vector.shape

(3064, 16825)

In [146]:
## 또는 아래와 같이 구할 수 도 있다. 답은 같다
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)

16528

In [147]:
# 테스트셋도 변환
X_test_dtm = vect.transform(X_test)
X_test_dtm

<1022x16528 sparse matrix of type '<class 'numpy.int64'>'
	with 45310 stored elements in Compressed Sparse Row format>

In [148]:
test_vector = pd.DataFrame(X_test_dtm.toarray(), columns=vect.get_feature_names())
test_vector.head()

,00,000,00a,00am,00pm,01,02,03,03342,04,...,zucchini,zuchinni,zumba,zupa,zuzu,zwiebel,zzed,éclairs,école,ém
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
test_vector.shape

(1022, 16528)

### 4-2) x_train 특정 x 벡터화 시 처음으로 1이 나오는 단어

In [150]:
## 해당 텍스트에 단어가 존재하면 벡터화 한 값이 1보다 클것임
##문제에서 1이라고 했으면 ==1 단어가 최초로 나왔다면 0보다 크다로
np.where(train_vector.loc[2]>0)
##np.where(train_vector.loc[2]==1)

(array([ 1312,  1444,  1610,  2838,  2856,  3704,  6497,  6712,  7162,
         7208,  7297,  7379,  8212,  8431,  8567,  8759,  8805,  9392,
         9880, 10676, 10853, 10994, 11833, 13888, 14324, 14987, 15629,
        16092], dtype=int64),)

In [151]:
train_vector.columns[563]

'afternoon'

### 4-3) love가 몇번째 피쳐인지?

In [152]:
## 왠지 index를 알수있는 method가 있을꺼 같은 느낌이 든다. dir 을 해보자

dir(test_vector.columns)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__inv__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmul__',
 '__rpow__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__unicode__',
 '__weakref__',
 '__xor__',
 '_accessors',
 '_add_comparison_methods',
 '_add_logical_methods',
 '_add_logical_methods_disabled',
 '_add_numeric_methods',
 '_add_num

In [153]:
## get_indexer가 왠지 인덱스를 가져올꺼같다

test_vector.columns.get_indexer(['love'])

array([8759], dtype=int64)

In [155]:
## 테스트해보자. 맞네 따라서 답은 8759
test_vector.columns[8759]

'love'

## 5.모델생성하기
  1) 문제에 설명한 모델 무엇인지 쓰기나이브베이즈(사전확률/우도/사후확률)기법으로 모델링
  2) 정확도 구하기
  3) confusion matrix 구하기
 


### 5-1) 문제에서 설명한 모델 무엇인지 쓰고 학습하기
문제에서 사전확률, 우도(likelihood) , 사후확률 어쩌구 라고 했다. -> 나이브베이즈

 베이즈 정리 : 사전확률과 우도(조건부확률)을 알고있다면 사후확률을 구할 수 있다.
 Naive Bayes : 베이즈 정리를 나이브하게 
 
 sklearn 에서 제공하는 나이브베이즈 분류기
 · GaussianNB : 연속적인 데이터에 적용
 · BernoulliNB : 이진 데이터, 이벤트 발생여부
 · MultinomialNB : 카운트데이터(ex> 단어횟수), 이벤트의 빈도수
 
  > MultinomialNB와 BernoulliNB는 모델 복잡도 조절을 위한 alpha 매개변수 존재. 
    alpha가 크면 완만해지고 모델복잡도 낮아짐.
    성능향상에 크게 기여하지 않으나 어느정도 정확도는 높일 수 있음
    

### 5-2) 정확도 구하기

In [156]:
## 이제 나이브베이즈 인지 알았으니, 학습하자
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [157]:
## 학습한 모델로 test 데이터 예측해보자
y_pred_class = nb.predict(X_test_dtm)
y_pred_class

array([5, 5, 5, ..., 5, 1, 5], dtype=int64)

In [158]:
# 예측값의 정확도 확인 (약 91.59%) -> 
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.9158512720156555

### 5-3) confusion matrix 구하기

In [159]:
# 오차행렬(confusion matrix) 확인
metrics.confusion_matrix(y_test, y_pred_class)

array([[124,  60],
       [ 26, 812]], dtype=int64)

## 6. 모델결과기반으로 더 자세히 물어보기
  1) 평점이 1인경우 중요한 단어 5개, 평점이 5인경우 중요한 단어 5개
  2) 비율이 높은거 5개

In [160]:
##모델의 클래스는 아래와 같음
nb.classes_

array([1, 5], dtype=int64)

In [161]:
# 행은 예측값, 열은 단어를 나타냅니다
nb.feature_count_.shape

(2, 16528)

In [162]:
nb.feature_count_

array([[26.,  4.,  1., ...,  0.,  0.,  0.],
       [39.,  5.,  0., ...,  1.,  1.,  1.]])

In [163]:
# 평점이 1인 경우 각 단어가 몇번 출현했는지 확인
stars1_token_count = nb.feature_count_[0, :]
stars1_token_count

array([26.,  4.,  1., ...,  0.,  0.,  0.])

In [164]:
# 평점이 5인 경우 각 단어가 몇번 출현했는지 확인
stars5_token_count = nb.feature_count_[1, :]
stars5_token_count

array([39.,  5.,  0., ...,  1.,  1.,  1.])

In [165]:
# create a DataFrame of tokens with their separate ham and spam counts
## X_train_tokens = 아까 위에서 저장한 벡터화한 컬럼 값
tokens = pd.DataFrame({'token':X_train_tokens, 'stars1':stars1_token_count, 'stars5':stars5_token_count}).set_index('token')
tokens.head()

,stars1,stars5
token,,
00,26.0,39.0
000,4.0,5.0
00a,1.0,0.0
00am,3.0,2.0
00pm,1.0,4.0


In [166]:
tokens.shape

(16528, 2)

In [167]:
dir(tokens)

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_SLICEMAP',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',


In [168]:
# examine 5 random DataFrame rows
tokens.sample(5, random_state=6)

,stars1,stars5
token,,
teams,1.0,2.0
lifter,0.0,1.0
decaf,0.0,1.0
airfair,0.0,1.0
consistancy,0.0,1.0


### 6-1) 평점이 1인경우 중요한 단어 5개, 평점이 5인경우 중요한 단어 5개

In [170]:
## 평점이 1인경우 중요한 단어 5개 - 평점 1 기준으로 sorting 한다 desc로, sort_values()가 안되면 -> sort()로
tokens.sort_values(by=['stars1'],axis=0,ascending =False)

,stars1,stars5
token,,
food,420.0,1241.0
place,385.0,1485.0
like,331.0,940.0
just,309.0,951.0
service,237.0,680.0
good,233.0,1235.0
time,217.0,768.0
don,189.0,464.0
didn,159.0,221.0


In [172]:
## 평점이 5인경우 중요한 단어 5개 - 평점 5 기준으로 sorting 한다 desc로, sort_values()가 안되면 -> sort()로
tokens.sort_values(by=['stars5'],axis=0,ascending =False)

,stars1,stars5
token,,
great,69.0,1501.0
place,385.0,1485.0
food,420.0,1241.0
good,233.0,1235.0
just,309.0,951.0
like,331.0,940.0
love,38.0,847.0
time,217.0,768.0
best,56.0,736.0


### 6-2) 비율이 높은거 5개
 이거 하기위해서 문제에서 어쩌구 저쩌구 시킨게 있다 그대로 해보자.
 
 먼저, 0으로 나누는 문제를 방지하기 위해 모든 행에 1씩 더하기

In [173]:
# 0으로 나누는 걸 방지하기 위해 모든 행에 1씩 더함 (smoothing)
tokens['stars1'] = tokens.stars1 + 1
tokens['stars5'] = tokens.stars5 + 1
tokens.sample(5, random_state=6)

,stars1,stars5
token,,
teams,2.0,3.0
lifter,1.0,2.0
decaf,1.0,2.0
airfair,1.0,2.0
consistancy,1.0,2.0


각 단어가 전체 평점5과/평점1 에서 등장한 비율을 계산해보자

In [177]:
## 평점 1과 5 별로 각 총 등장 건수가 어떤지 확인
nb.class_count_

array([ 565., 2499.])

In [178]:
# 비율로 변환
# 
tokens['stars1'] = tokens.stars1 / nb.class_count_[0] ## 평점1 total count 로 각각 나눔
tokens['stars5'] = tokens.stars5 / nb.class_count_[1] ## 평점5 total count 로 각각 나눔
tokens.sample(5, random_state=6) ## 샘플보기

,stars1,stars5
token,,
teams,0.000006,4.803842e-07
lifter,0.000003,3.202562e-07
decaf,0.000003,3.202562e-07
airfair,0.000003,3.202562e-07
consistancy,0.000003,3.202562e-07


In [183]:
# 최종 비율 계산
tokens['star_ratio'] = tokens.stars5 / tokens.stars1
tokens.sample(5, random_state=6)

,stars1,stars5,star_ratio
token,,,
teams,0.000006,4.803842e-07,0.076675
lifter,0.000003,3.202562e-07,0.102234
decaf,0.000003,3.202562e-07,0.102234
airfair,0.000003,3.202562e-07,0.102234
consistancy,0.000003,3.202562e-07,0.102234


In [185]:
## 비율 기준으로 sorting 한다 desc로, sort_values()가 안되면 -> sort()로
tokens.sort_values(by=['star_ratio'],axis=0,ascending =False).head(10)

,stars1,stars5,star_ratio
token,,,
fantastic,0.000006,0.000031,4.932779
perfect,0.000009,0.000039,4.174546
yum,0.000003,0.000010,3.169247
favorite,0.000022,0.000055,2.519332
outstanding,0.000003,0.000008,2.504727
brunch,0.000003,0.000007,2.146909
gem,0.000003,0.000006,2.044675
mozzarella,0.000003,0.000006,1.993559
pasty,0.000003,0.000006,1.993559


## 7. 전체 데이터로 모델만들기
 1000개만 가지고 하라는 옵션이 있었던거 같은데...
 F1 스코어로 판단한다고 했음
  ※ F1 score는 데이터 label이 불균형 구조일 때, 모델의 성능을 정확하게 평가

In [186]:
X = df['text']
y = df['stars']

In [187]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7500,)
(2500,)
(7500,)
(2500,)


In [217]:
CountVectorizer?

In [189]:
# CountVectorizer 로드
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words='english')

##vect = CountVectorizer(max_features=1000)



In [190]:
## 데이터 벡터로 변환
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [191]:
## DataFrame 형태로 변환한다
train_vector = pd.DataFrame(X_train_dtm.toarray(), columns=vect.get_feature_names())
train_vector.head()


,00,000,007,00a,00am,00pm,01,02,04,05,...,zur,zuzu,zuzus,zweigel,zwiebel,zy,zzed,zzzzzzzzzzzzzzzzz,école,òc
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [192]:
# 테스트셋도 변환
X_test_dtm = vect.transform(X_test)
test_vector = pd.DataFrame(X_test_dtm.toarray(), columns=vect.get_feature_names())
test_vector.head()

,00,000,007,00a,00am,00pm,01,02,04,05,...,zur,zuzu,zuzus,zweigel,zwiebel,zy,zzed,zzzzzzzzzzzzzzzzz,école,òc
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 먼저, 나이브베이즈 해보자
 아까 위에서한거 참조

In [201]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)

## 학습한 모델로 test 데이터 예측해보자
y_pred_class = nb.predict(X_test_dtm)
print(y_pred_class)

# 예측값의 정확도 및 f1 score 구하기 -> 
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))
print(metrics.f1_score(y_test, y_pred_class,average=None))


[4 4 5 ... 4 5 5]
0.4672
[0.3902439  0.14685315 0.15102041 0.53654415 0.53375796]


#### Logistic 해보자.

우리가 해야하는건 Logistic 분류! sklearn 사용시 여기서 유의해야하는점!!!
분류 알고리즘 : LogisticRegression 이다. (Regression이 있다고 혼동하면안됨)


>>> from sklearn.datasets import load_iris
>>> from sklearn.linear_model import LogisticRegression
>>> X, y = load_iris(return_X_y=True)
>>> clf = LogisticRegression(random_state=0, solver='lbfgs',
...                          multi_class='multinomial').fit(X, y)
>>> clf.predict(X[:2, :])
array([0, 0])
>>> clf.predict_proba(X[:2, :]) 
array([[9.8...e-01, 1.8...e-02, 1.4...e-08],
       [9.7...e-01, 2.8...e-02, ...e-08]])
>>> clf.score(X, y)
0.97...

In [210]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression() 
clf.fit(X_train_dtm, y_train)
## 로지스틱인데 0,1 두 경우만 되는거 아니야? 이건 라벨이 멀티인데... 라고 생각할수 있다.  
## ? 해서 보면, multi_class 라는 옵션을 줄 수 도 있고 아무것도 안넣어도 멀티class 분석 해 준다

## 학습한 모델로 test 데이터 예측해보자
y_pred_class = clf.predict(X_test_dtm)
print(y_pred_class)

# 예측값의 정확도 확인  -> 
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))
print(metrics.f1_score(y_test, y_pred_class,average=None))


[4 5 5 ... 4 5 3]
0.4944
[0.45859873 0.35532995 0.27956989 0.52337453 0.57977528]


In [221]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='sag',multi_class='multinomial') 
clf.fit(X_train_dtm, y_train)
## 이건 옵션 추가한거 멀티 옵션 추가시 솔버를 지정해야한다고 써있음 


## 학습한 모델로 test 데이터 예측해보자
y_pred_class = clf.predict(X_test_dtm)
print(y_pred_class)

# 예측값의 정확도 확인 -> 
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))
print(metrics.f1_score(y_test, y_pred_class,average=None))


[4 5 5 ... 4 5 4]
0.5052
[0.48823529 0.37558685 0.30149254 0.53398586 0.58898551]


C:\Users\YOUNG\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [231]:
## 이 결과가 제일 좋음. 이 결과 값을 파일로 내리자

np.savetxt('data/output.txt',y_pred_class)

In [212]:
LogisticRegression?

이번엔 DecisionTree 를 해볼까 한다

>>> from sklearn.datasets import load_iris
>>> from sklearn.model_selection import cross_val_score
>>> from sklearn.tree import DecisionTreeClassifier
>>> clf = DecisionTreeClassifier(random_state=0)
>>> iris = load_iris()
>>> cross_val_score(clf, iris.data, iris.target, cv=10)

In [216]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X_train_dtm, y_train)



## 학습한 모델로 test 데이터 예측해보자
y_pred_class = clf.predict(X_test_dtm)
print(y_pred_class)

# 예측값의 정확도 확인 -> 
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))
print(metrics.f1_score(y_test, y_pred_class,average=None))


[2 5 5 ... 4 5 2]
0.3816
[0.32727273 0.17672414 0.22510823 0.41713641 0.47743468]


In [ ]:
마지막으로 랜덤포레스트도 한번 해보자

>>> from sklearn.ensemble import RandomForestClassifier
>>> from sklearn.datasets import make_classification

>>> X, y = make_classification(n_samples=1000, n_features=4,
...                            n_informative=2, n_redundant=0,
...                            random_state=0, shuffle=False)
>>> clf = RandomForestClassifier(n_estimators=100, max_depth=2,
...                              random_state=0)
>>> clf.fit(X, y)  
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
>>> print(clf.feature_importances_)
[0.14205973 0.76664038 0.0282433  0.06305659]

In [220]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
clf.fit(X_train_dtm, y_train)



## 학습한 모델로 test 데이터 예측해보자
y_pred_class = clf.predict(X_test_dtm)
print(y_pred_class)

# 예측값의 정확도 확인  -> 
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))
print(metrics.f1_score(y_test, y_pred_class,average=None))

## 결과가 이상하네..

[4 4 4 ... 4 4 4]
0.3592
[0.         0.         0.         0.50873908 0.16403162]


C:\Users\YOUNG\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
